In [1]:
!pip install booknlp
!pip install transformers==4.30.0
!python -m spacy download en_core_web_sm

!wget https://www.gutenberg.org/files/1342/1342-0.txt -O pride_and_prejudice.txt


from booknlp.booknlp import BookNLP
import json
import os

from collections import defaultdict
import multiprocessing as mp

class SequentialCharacterTracker:
    def __init__(self):
        self.model_params = {
            "pipeline": "entity,quote,supersense,event,coref",
            "model": "big"
        }
        self.booknlp = BookNLP("en", self.model_params)

    def process_book(self, input_file, output_dir, book_id):
        """Process the book using BookNLP"""
        os.makedirs(output_dir, exist_ok=True)
        self.booknlp.process(input_file, output_dir, book_id)

    def get_canonical_names(self, book_data):
        """Create mapping of character IDs to their main names"""
        canonical_names = {}
        for character in book_data["characters"]:
            proper_mentions = character["mentions"]["proper"]
            if proper_mentions:
                main_name = max(proper_mentions, key=lambda x: x['c'])['n']
                canonical_names[character["id"]] = main_name
        return canonical_names

    def track_sequential_mentions(self, input_file, output_dir, book_id):
        """Create chronological tracking of character mentions"""
        self.process_book(input_file, output_dir, book_id)

        with open(f"{output_dir}{book_id}.book", 'r', encoding='utf-8') as f:
            book_data = json.load(f)

        canonical_names = self.get_canonical_names(book_data)

        entities_file = f"{output_dir}{book_id}.entities"
        tokens_file = f"{output_dir}{book_id}.tokens"

        chronological_mentions = self.process_entities(entities_file, tokens_file, canonical_names)

        self.write_output(chronological_mentions, f"{output_dir}sequential_mentions.txt")

    def process_entities(self, entities_file, tokens_file, canonical_names):
        """Process entities file and create chronological mentions"""
        chronological_mentions = defaultdict(lambda: defaultdict(list))

        with open(entities_file, 'r', encoding='utf-8') as entities_f, \
             open(tokens_file, 'r', encoding='utf-8') as tokens_f:
            next(entities_f)  # Skip header
            next(tokens_f)  # Skip header

            entities_lines = entities_f.readlines()
            tokens_lines = tokens_f.readlines()

            for entity_line in entities_lines:
                parts = entity_line.strip().split('\t')
                if len(parts) >= 6:
                    coref_id = int(parts[0])
                    start_token = int(parts[1])
                    mention_text = parts[5]

                    if coref_id in canonical_names:
                        token_line = tokens_lines[start_token]
                        token_parts = token_line.strip().split('\t')
                        paragraph_num = int(token_parts[0])
                        sentence_num = int(token_parts[1])

                        chronological_mentions[paragraph_num][sentence_num].append({
                            'character': canonical_names[coref_id],
                            'mention': mention_text
                        })

        return chronological_mentions

    def write_output(self, chronological_mentions, output_file):
        """Write the chronological mentions to the output file"""
        with open(output_file, 'w', encoding='utf-8') as f:
            for paragraph, sentences in sorted(chronological_mentions.items()):
                f.write(f"\n[Paragraph {paragraph}]\n")
                for sentence, mentions in sorted(sentences.items()):
                    f.write(f"[Sentence {sentence}]: ")
                    for mention in mentions:
                        if mention['mention']==mention['character']:
                          f.write(f"{mention['mention']}, ")
                        else:
                          f.write(f"{mention['mention']}[{mention['character']}], ")
                    f.write("\n")

if __name__ == "__main__":
    tracker = SequentialCharacterTracker()
    tracker.track_sequential_mentions(
        input_file="pride_and_prejudice.txt",
        output_dir="pride_and_prejudice/",
        book_id="pride_and_prejudice"
    )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.30.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/270M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/booknlp/english/bert_qa.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(modelFile, map_loc

--- startup: 97.297 seconds ---
--- spacy: 21.492 seconds ---
--- entities: 129.385 seconds ---
--- quotes: 0.270 seconds ---
--- attribution: 48.895 seconds ---
--- name coref: 0.535 seconds ---
--- coref: 53.557 seconds ---
--- TOTAL (excl. startup): 254.890 seconds ---, 152569 words
